In [1]:
from os import listdir
import jsonlines
import itertools
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import pymorphy2 as pm
import re
from bisect import bisect_left
#from joblib import Parallel, delayed
from collections import Counter
from tqdm import trange
from tqdm import tqdm
from math import log
from random import shuffle
from sys import exit
import numpy as np
import json
import networkx as nx


from mediawiki import MediaWiki
from mediawiki import PageError
from mediawiki import MediaWikiCategoryTreeError

stopwords_ru = set(stopwords.words('russian'))
tknzr = TweetTokenizer()
morph = pm.MorphAnalyzer()
wiki = MediaWiki(url='http://ru.wikipedia.org/w/api.php')

In [2]:
path = listdir('c')
path = sorted(path)[:9]
#path = ['AA', 'AD', 'AE']
print(path)

data = []
for p in tqdm(path):
    with jsonlines.open('c/' + p, 'r') as f:
        for entry in f:
            data.append(entry)

['AA', 'AB', 'AD', 'AE', 'AF', 'AI', 'AJ', 'AK', 'AQ']


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:08<00:00,  1.05it/s]


# Разбираемся с категориями. Добавялем категории, который есть в ok, но которых нет в дереве/dag

In [3]:
cat_graph = nx.read_gpickle('sources/graph_gpickle.gpickle')

In [5]:
links = [link for link in jsonlines.open('links/link_directed.txt', 'r')]
categories_id_dict = {}
id_categories_dict = {}
for link in tqdm(links):
    id_categories_dict[link['node_1_id']] = re.sub(r'^Категория:', '', link['node_1_title'])
    id_categories_dict[link['node_2_id']] = re.sub(r'^Категория:', '', link['node_2_title'])
    
    categories_id_dict[re.sub(r'^Категория:', '', link['node_1_title'])] = link['node_1_id']
    categories_id_dict[re.sub(r'^Категория:', '', link['node_2_title'])] = link['node_2_id']


100%|██████████████████████████████████████████████████████████████████████| 719813/719813 [00:06<00:00, 103830.36it/s]


In [6]:
for i in trange(len(data)):
    categories = data[i]['categories']
    categories_nu = set()
    for c in categories:
        c = re.sub(r'\u200e', '', c)
        c = re.sub(r'\xa0', ' ', c)
        categories_nu.add(re.sub(r'\|.*', '', c))
    data[i]['categories'] = categories_nu
    
categories = [d['categories'] for d in data]    
categories = set(list(itertools.chain(*categories)))

categories_dict = {c: 0 for c in categories}
for d in data:
    for c in d['categories']:
        if categories_dict.get(c) is not None:
            categories_dict[c] += 1
            
ok_set = {c for c in categories if categories_dict[c] > 5 and categories_dict[c] < 5000}

100%|███████████████████████████████████████████████████████████████████████| 116584/116584 [00:01<00:00, 73219.49it/s]


In [7]:
missed = []
cat_graph_nodes = {id_categories_dict[cat] for cat in cat_graph.nodes()}
for c in tqdm(ok_set):
    if c not in cat_graph_nodes:
        missed.append(c)

len(missed)

100%|███████████████████████████████████████████████████████████████████████| 14857/14857 [00:00<00:00, 1058227.33it/s]


557

In [10]:
doomed = set()
restored = set()

for miss in tqdm(missed):
    flag = True
    while flag:
        try:
            res = wiki.categorytree(category=miss, depth=1)
            flag = False
        except PageError:
            try:
                res = wiki.categorytree(category='Категория:'+miss, depth=1)
                flag = False
            except PageError:
                doomed.add(miss)
                flag = False
        except MediaWikiCategoryTreeError:
            sleep(3)
            
    parents = res[list(res.keys())[0]]['parent-categories']
    parents = [re.sub(r'^Категория:', '', p) for p in parents]
    for parent in parents:
        if parent in cat_graph_nodes:
            flag = True
            new_node_id = max(cat_graph.nodes()) + 1
            cat_graph.add_edge(categories_id_dict[parent], new_node_id)
            id_categories_dict[new_node_id] = miss
            categories_id_dict[miss] = new_node_id
            restored.add((parent, miss))
            
    if not flag:
        doomed.add(miss)


len(doomed)

100%|████████████████████████████████████████████████████████████████████████████████| 557/557 [15:44<00:00,  1.70s/it]


160

In [12]:
with open("sources/id_cat.json", mode="w") as output:
    output.write(json.dumps(id_categories_dict))

In [14]:
with open("sources/cat_id.json", mode="w") as output:
    output.write(json.dumps(categories_id_dict))

In [16]:
ok_set = {c for c in ok_set if c not in doomed}
ok = sorted(ok_set)
with open("sources/accepted_categories.txt", mode="w", encoding="utf-8") as output:
    for c in ok:
        output.write(c + '\n')

article_category_dict = {}
for d in tqdm(data):
    cats = [c for c in d['categories'] if c in ok_set]
    if len(cats) > 0:
        article_category_dict[d['id']] = cats
with open("sources/article_cat.json", mode="w") as output:
    output.write(json.dumps(article_category_dict))

category_article_dict = {c: [d['id'] for d in data if c in d['categories']] for c in tqdm(ok_set)}
with open("sources/cat_article.json", mode="w") as output:
    output.write(json.dumps(category_article_dict))
    
nx.write_gpickle(cat_graph, 'sources/graph_gpickle.gpickle')

100%|████████████████████████████████████████████████████████████████████████████| 14697/14697 [03:24<00:00, 72.02it/s]


# Формируем тексты (лемматизированные и просто "очищенные")

In [11]:
words = []
ids = article_category_dict.keys()

with jsonlines.open('sources/texts.jl', 'w') as output:
    for d in tqdm(data):
        if d['id'] in ids:
            text = re.sub(r'\n+', '. ', d['text'])
            text = re.sub(r'\xa0', ' ', text)
            tokens = tknzr.tokenize(text=text)
            tokens = [t.lower() for t in tokens if t.lower() not in stopwords_ru]     #  if t.isalnum()
            text = ' '.join(tokens)
            output.write({'text': text, 'id': d['id']})
            words.extend(sorted(set([t for t in tokens if t.isalpha()])))

print("Done")
words = set(words)

100%|█████████████████████████████████████████████████████████████████████████| 116584/116584 [05:07<00:00, 378.72it/s]


Done


In [12]:
normal_forms = {}
for w in tqdm(words):
    nf = morph.parse(w)[0].normal_form
    normal_forms[w] = nf
    
with open("sources/normal_forms.json", mode="w") as output:
    output.write(json.dumps(normal_forms))

100%|██████████████████████████████████████████████████████████████████████| 1231449/1231449 [06:24<00:00, 3200.89it/s]


In [13]:
with jsonlines.open('sources/texts.jl', 'r') as file:
    with jsonlines.open('sources/normalized_texts.jl', 'w') as output:
        for f in tqdm(file):
            id = f['id']
            text = f['text']
            tokens = re.split(r' ', text)
            tokens = [normal_forms.get(t) for t in tokens if normal_forms.get(t) is not None]
            text = ' '.join(tokens)
            output.write({'text': text, 'id': id})

96794it [01:02, 1545.66it/s]


# Формируем tf-idf и svd

In [ ]:
texts = {item['id']: item['text'] for item in jsonlines.open('sources/normalized_texts.jl', 'r')}

In [ ]:
vectorizer = TfidfVectorizer(min_df=5, max_df=0.98, )
X_tfidf = vectorizer.fit_transform([text for (id, text) in sorted(texts.items())])
np.save("sources/tf_idf", X_tfidf)

In [ ]:
svd = TruncatedSVD(n_components=500, random_state=27)
X_svd = svd.fit_transform(text_tfidf)
np.save("sources/svd")